In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
#import seaborn as sns
import tensorflow as tf

In [12]:
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

In [8]:
import tarfile

In [13]:
# Set seed for experiment reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [7]:
!dir

drive  sample_data


In [3]:
PATH = './drive/MyDrive/ml_dataset/speech_commands_v0.01.tar.gz'

# Extract data from drive

In [9]:
if PATH.endswith("tar.gz"):
    tar = tarfile.open(PATH, "r:gz")
    tar.extractall('./data/')
    tar.close()

In [ ]:
!ls data/cat/

In [14]:
data_dir = pathlib.Path('./data/')

In [26]:
commands = np.array(tf.io.gfile.listdir(str(data_dir)))
commands = [x for x in commands if x not in {'LICENSE', 'README.md', 'validation_list.txt', 'testing_list.txt'}]
print(commands)
print(len(commands))

['cat', 'two', 'marvin', 'down', 'yes', 'eight', 'right', 'stop', 'one', 'off', 'go', 'left', 'wow', 'up', 'three', 'six', '_background_noise_', 'on', 'house', 'dog', 'nine', 'happy', 'seven', 'bed', 'no', 'four', 'tree', 'bird', 'sheila', 'zero', 'five']
31


# https://www.tensorflow.org/tutorials/audio/simple_audio
Extract the audio files into a list and shuffle it.
Split the files into training, validation and test sets using a 80:10:10 ratio, respectively.
